## Setup
Install accompanying pinon library. It is better to install in dev mode in the conda environment with `pip install -e .` Jupyter server must restart to see changes to conda environment.
Auto reload the pinon module.  Other imports may be ignored for auto reload e.g.  `%aimport -simfin`. Note use of option 2 for auto reload and excluding other modules, rather than option 1 and including only reloadable modules.  This is because option 1 does not work correctly with aliased imports.

In [17]:
# !pip install -e ..

%load_ext autoreload
%autoreload 2
%aimport -pandas
%aimport -simfin
%aimport -os

%aimport pinon

%matplotlib inline

# imports
import pandas as pd
import os

import pinon as pn
import simfin as sf
import simfin.names as sf_cols  # column name shortcuts

config = pn.Config('master')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Config file found at: /Users/daved/Dev/pinon/config/master.xlsx
Simfin data directory: /Users/daved/Dev/pinon/simfin_data


In [18]:
ts = config.parse_target_sheet('V', 10)
ts

Dataset "us-companies" on disk (2 days old).
- Loading from disk ... Done!


/Users/daved/opt/anaconda3/envs/py3_10_8_pinon/lib/python3.10/site-packages/simfin/load.py:144: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


,Company Name,IndustryId,Past Years Requested,Peer List
Ticker,,,,
AXP,AMERICAN EXPRESS CO,104007.0,10,[]
DFS,Discover Financial Services,104007.0,10,[]
MA,Mastercard Inc,104007.0,10,[]


In [5]:
c = pn.Companies()

In [7]:
res = c.get_company('MSFT')
res

SimFinId                 59265
Company Name    MICROSOFT CORP
IndustryId            101003.0
Name: MSFT, dtype: object